In [19]:
!pip install colorama

**IMPORTS**

In [20]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from google.colab import files
from matplotlib import pyplot
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
# modulo de cor:
import colorama
from colorama import Fore
from colorama import Style

 **1. FUNÇÕES**

In [21]:
'''Funções Estatísticas'''
def acuracia(tp,tn,total):
  return np.round((tp+tn)/total,4)
def precisao(tp,fp):
  return np.round(tp/(tp+fp),4)
def sensibilidade(tp,fn): #Sensibilidade
  return np.round(tp/(tp+fn),4)
def f1_score(tp,fp,fn):
  p = precisao(tp,fp)
  r = sensibilidade(tp,fn)
  return np.round((2 * p * r) / (p + r),4)
def step_decay_schedule(initial_lr, decay_factor, step_size):
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    return LearningRateScheduler(schedule)

In [22]:
'''Função de Carregamento dos arquivos'''
def load_data(tables_path):
  lista_temp = os.listdir(tables_path) 
  lista_temp.sort() # Ordena a lista com os nomes dos arquivos;
  lista_testes = []
  for i in range(10):
    temp = str(lista_temp[i])
    # Abrindo o arquivo da temporada e convertendo em DataFrame:
    datasetTemporada = pd.read_csv(tables_path+temp)
    df = pd.DataFrame(datasetTemporada)
    #Removendo coluna 'Time'(Tempo, em Inglês): (Atualizado em: 24_02_2021)
    lista_colunas = df.columns.values.tolist()
    if lista_colunas[2] == 'Time':
        df = df.drop(['Time'],axis=1)
    #Corrigindo diferença de quantidade de colunas: (Atualizado em: 24_02_2021)
    columns_size = df.shape[1]
    if columns_size < 105 and columns_size > 26:
        diff = 105 - columns_size
        j = 0 
        name_column = ''
        while j < diff:
            df[name_column] = ""
            name_column = name_column+"-"
            j = j + 1
    # Filtra o DataFrame original com apenas as colunas necessárias para análise;
    odds = np.arange(26,105) # atribuindo o intervalo das colunas 25 a 70, onde estão as odds não analisadas. (retorna valores espaçados igualmente dentro de um intervalo definido)
    df_filtrado = df.drop(df.columns[odds], axis = 1, inplace = True)  # Removendo as colunas de odds (permanentemente do DF original)
    df_filtrado = df.drop(columns = ['Div','Date','HTHG','HTAG','HTR','HF','AF','HC',
                                'AC','HY','AY','HR','AR','Referee','B365D','B365A']) # Adicionei à remoção: B365D,B365A e as colunas dos times(Atualizado em: 25_02_2021)
    if len(df_filtrado) > 380:  # Corrigindo uma linha fantasma que surge no df, quando importa por google drive!
        df_filtrado = df_filtrado[:-1]
    # Adicionando a coluna oddLayH no DF "menor";
    oddsLayH = []
    for k in range(380):
        oddH = float(df_filtrado.iloc[k][9])
        oddLayH = 1/(1-(1/oddH))
        oddsLayH.append(np.round(oddLayH,2))
    df_filtrado['oddLayH'] = oddsLayH
    # Convertendo resultados de FTR para binario(Vitoria=1, Derrota=0 ,Empate=0) no DF "menor";
    listR = []
    for l in range(380): 
        if df_filtrado.iloc[l,4] == 'H':
            listR.append(1)
        else:
            listR.append(0)
    df_filtrado['FTR'] = listR
    # Reposicionando as colunas:
    df_filtrado = df_filtrado[['HomeTeam','AwayTeam','FTHG','FTAG','HS','AS','HST','AST','FTR','B365H','oddLayH']]
    # Renomeando colunas B365H:
    df_filtrado.rename(columns = {'B365H':'oddH','HomeTeam':'Mandante','AwayTeam':'Visitante','FTHG':'GPM','FTAG':'GPV','HS':'CM','AS':'CV','HST':'CAAM','AST':'CAAV','FTR':'Resultado'}, inplace = True)
    lista_testes.append(df_filtrado)
  return lista_testes

In [23]:
'''Função que gera novos dataframes com as médias de cada temporada, calculadas rodada a rodada,excluindo-se as 4 rodadas iniciais:'''
def gera_df_medias_temporada(df_temporada):
  temp = df_temporada
  rodada = 5
  novos_dfs_rodadas = []
  df_temp = pd.DataFrame()
  while rodada <= 38:
    #pula as X linhas iniciais,para gerar médias:
    rodada_teste = rodada
    max = (rodada_teste*10) - 10
    df = temp.iloc[0:max,0:8] # Gera um DataFrame com todas as linhas antes da rodada atual. A media sera calcula apenas com os valores anterior a rodada.
    # Coleta os times do campeonato:
    times = {}
    for i in range(40):
      time = df.iloc[i][0]
      if time in times:
        pass
      else:
        times[time] = []
    # Colentado as médias de cada time, para a rodada especificada:
    for nome_do_time in times.keys():
      '''Mandante'''
      temp_df = df.query('Mandante=="'+nome_do_time+'"')
      media_GPM = np.round(temp_df['GPM'].mean(),2)
      media_CM = np.round(temp_df['CM'].mean(),2)
      media_CAAM = np.round(temp_df['CAAM'].mean(),2)
      '''Visitante'''
      temp_df = df.query('Visitante=="'+nome_do_time+'"')
      media_GPV = np.round(temp_df['GPV'].mean(),2)
      media_CV = np.round(temp_df['CV'].mean(),2)
      media_CAAV = np.round(temp_df['CAAV'].mean(),2)

      times[nome_do_time].append(media_GPM)
      times[nome_do_time].append(media_GPV)
      times[nome_do_time].append(media_CM)
      times[nome_do_time].append(media_CV)
      times[nome_do_time].append(media_CAAM)
      times[nome_do_time].append(media_CAAV)
    # Gerando novo dataFrame com as medias da rodada específica:
    min = (rodada*10)-10
    max = min + 9
    df = temp
    df_copy = pd.DataFrame.copy(df,deep=True)  # Deep: se for True, as alterações feitas na cópia não transferem para o original;
    df_copy['GPM'] = df_copy['GPM'].astype(float)
    df_copy['GPV'] = df_copy['GPV'].astype(float)
    df_copy['CM'] = df_copy['CM'].astype(float)
    df_copy['CV'] = df_copy['CV'].astype(float)
    df_copy['CAAM'] = df_copy['CAAM'].astype(float)
    df_copy['CAAV'] = df_copy['CAAV'].astype(float)
    # Substituindo o dataframe original pelas medias de cada atributo:
    dic = times
    for time in times.keys():
      reqd_Index = df[df['Mandante']==time].index.tolist() # Encontra todos os indices onde o time é Mandante;
      for i in reqd_Index:
        if i >= min and i < min+10:
          df_copy.iat[i,2] = np.round(dic[time][0],2)
          df_copy.iat[i,4] = np.round(dic[time][2],2)
          df_copy.iat[i,6] = np.round(dic[time][4],2)
      reqd_Index = df[df['Visitante']==time].index.tolist() # Encontra todos os indices onde o time é Visitante;
      for i in reqd_Index:
        if i >= min and i < min+10:
          df_copy.iat[i,3] = np.round(dic[time][1],2)
          df_copy.iat[i,5] = np.round(dic[time][3],2)
          df_copy.iat[i,7] = np.round(dic[time][5],2)
    df_copy.rename(columns = {'GPM':'MediaGPM','GPV':'MediaGPV','CM':'MediaCM','CV':'MediaCV','CAAM':'MediaCAAM','CAAV':'MediaCAAV'}, inplace = True)
    df_medias_rodada_treino = df_copy[min:max+1]
    novos_dfs_rodadas.append(df_medias_rodada_treino)
    '''print(Fore.GREEN+Style.BRIGHT+"RODADA: %d" % rodada+Style.RESET_ALL)
    display(df_medias_rodada)
    print('\n')
    print('\n')'''
    rodada += 1
  df_medias_temporada = df_temp.append(novos_dfs_rodadas,ignore_index=True)
  return df_medias_temporada

''' Função que armazena todos os dataframes com médias de cada temporada em uma lista '''
def gera_lista_medias_temporadas(dfs):  
  # Lista com os dfs das temproadas com as medias, rodada a rodada.
  dfs_medias_temporadas = []
  for df in dfs:
    temp_df = gera_df_medias_temporada(df)
    dfs_medias_temporadas.append(temp_df)
  return dfs_medias_temporadas

'''Função que retorna os dataframes concatenados dos treinos base,armazenados em uma lista'''
def gera_lista_temporadas_treino_base(lista_medias_temporadas):
  # Concatena temporadas treino base:
  lista = lista_medias_temporadas
  i = 0
  base_01 = 0
  base_02 = 1
  base_03 = 2
  datasets_de_treinos_iniciais = []
  while i < 7:
    df_01 = lista[base_01]
    df_02 = lista[base_02]
    df_03 = lista[base_03]
    temp_df = pd.concat([df_01,df_02,df_03],ignore_index=True)
    datasets_de_treinos_iniciais.append(temp_df)
    ##### Iteração:
    base_01 += 1
    base_02 += 1
    base_03 += 1
    i += 1
  return datasets_de_treinos_iniciais

'''Método que retorna um dataframe com médias da rodada a ser preditada'''
def gera_novo_df_com_medias_da_rodada_teste(rodada,dataFrame):
  min = (rodada - 5) * 10
  max = min + 10
  return dataFrame.iloc[min:max,0:11]

'''Método que junta o dataframe com médias da rodada ao treino com médias'''
def concatena_rodada_teste_ao_treino(df_treino,df_medias_rodada_teste):
  size = len(df_treino)
  temp_df = pd.concat([df_treino,df_medias_rodada_teste], ignore_index=True)
  size += 10
  temp_df.index = range(size)
  return temp_df

def concatena_rodada_teste_a_rodada_anterior(df_anterior,df_rodada_teste):
  size = len(df_anterior)
  temp_df = pd.concat([df_anterior, df_rodada_teste], ignore_index=True)
  size += 10
  temp_df.index = range(size)
  return temp_df

'''Méotodo que prepara o dataframe para ser inserido como dataset X na Rede Neural; serve para treino e para teste'''
def cria_X(dataFrame):
  temp_df = dataFrame.drop(columns = ['Mandante', 'Visitante', 'Resultado','oddLayH'])
  return temp_df

'''Méotodo que prepara o dataframe para ser inserido como dataset Y na Rede neural; serve para treino e para teste'''
def cria_Y(dataFrame):
  temp_df = dataFrame.drop(columns = ['Mandante', 'Visitante','MediaGPM','MediaGPV','MediaCM','MediaCV','MediaCAAM','MediaCAAV','oddH','oddLayH'])
  return temp_df
  
'''Méotodo que prepara o df da rodada para ser predições, com apenas as odds e o resultado da partida'''
def cria_base_predicao_da_rodada(dataFrame):
  temp_df = dataFrame.drop(columns = ['Mandante', 'Visitante','MediaGPM','MediaGPV','MediaCM','MediaCV','MediaCAAM','MediaCAAV'])
  return temp_df

**2.EXECUÇÃO**

In [24]:
path = '/content/drive/MyDrive/Iniciação Científica - UAEst - Prof. Alexsandro_2019.2/Pesquisa_Caio Medeiros/Projeto 2020_2021_Deep Learning/Tabelas - Premier League/Tabelas Premier League/'
dfs = load_data(path)

lista_medias_temporadas = gera_lista_medias_temporadas(dfs) # Lista de todas as temporadas com médias
lista_treinos_base = gera_lista_temporadas_treino_base(lista_medias_temporadas) # Lista de todos os treinos base (7 elementos)

**ENTRADA DE PARÂMETROS**

In [25]:
 #neuronios: 64 | 128
 #camadas:   10 | 4 & 8

In [27]:
neuronios = 64
camadas = 10

**EXECUÇÃO DA REDE**

In [29]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Dense(neuronios, input_dim = 7, kernel_initializer = 'uniform', activation = 'relu'))
for i in range(camadas):
  if i == 0:
    model.add(tf.keras.layers.Dense(neuronios, input_dim = 7, kernel_initializer = 'uniform', activation = 'relu'))
    if camadas == 1:
      break
  else:
    model.add(tf.keras.layers.Dense(neuronios, activation = 'relu'))


model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer = opt, loss='binary_crossentropy', metrics=['accuracy'])
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose = 0, save_best_only=True)

temporada = 2013
teste = 3
treino_base = 0
proporcao = []

sugere_vitoria = 0
sugere_contra = 0
real_vitoria = 0
real_contra = 0
sugestao_vitoria_mandante = []
real_vitoria_mandante = []

prob_vitoria_mandante = []
prob_contra_mandante = []
probabilidades = []
while temporada < 2015 and treino_base < 7:

  if temporada > 2013:
    saved_model = load_model('best_model.h5')
    saved_model.compile(optimizer = opt, loss='binary_crossentropy', metrics=['accuracy'])
    mc = ModelCheckpoint('best_model.h5', monitor = 'val_loss', mode='min', verbose = 0, save_best_only=True)

  temp_df = lista_treinos_base[treino_base] # vai do 0 ao 6 (7 treinos ao total)
  temp_df_02 = pd.DataFrame()
  rodada = 5
  ganhos = 0
  perdas = 0

  while rodada <= 38:

    '''if rodada > 5:
      saved_model = load_model('best_model.h5')
      saved_model.compile(tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
      mc = ModelCheckpoint('best_model.h5', monitor = 'val_loss', mode='min', verbose = 0, save_best_only=True)'''
    
    # Montando os datasets de Treino X e Y (começará a partir dar 4ª round e terá médias em vez de valores base, com as 50 primeira linhas depois das 30 iniciais):
    data_x_train = cria_X(temp_df)
    data_y_train = cria_Y(temp_df)        
    x_train = tf.convert_to_tensor(data_x_train, dtype = tf.int32)  
    y_train = tf.convert_to_tensor(data_y_train, dtype = tf.int32)
    # Montando os datasets de teste X e Y:
    df_rodada_teste = gera_novo_df_com_medias_da_rodada_teste(rodada,lista_medias_temporadas[teste])
    data_x_test = cria_X(df_rodada_teste)
    data_y_test = cria_Y(df_rodada_teste)
    x_test = tf.convert_to_tensor(data_x_test, dtype = tf.float32)  
    y_test = tf.convert_to_tensor(data_y_test, dtype = tf.float32)  

    # Treinando o modelo sem/com parada antecipada e/ou decaimento:
    lr_sched = step_decay_schedule(initial_lr = 0.0001, decay_factor = 0.25, step_size = 10)
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose = 0, patience = 17)
    model.fit(x_train, y_train, validation_split = 0.2, epochs = 100, callbacks =[es,mc], verbose = 0)  # Analisar os parametros possiveis. Verificar Sobreajuste e Subajuste
    # Analisando as predições e o Lucro/prejuízo:
    df_base_predicoes = cria_base_predicao_da_rodada(df_rodada_teste)
    rna_predicoes = model.predict(x_test)
    
    df_base_predicoes_02 = df_base_predicoes
    df_base_predicoes_02['Probabilidade'] = np.round(rna_predicoes,4)
    
    l_p_lista = []
    
    for i in range(10):
      prob = rna_predicoes[i][0]
      oddH = df_base_predicoes.iloc[i][1]
      oddLayH = df_base_predicoes.iloc[i][2]
      result = df_base_predicoes.iloc[i][0]
      
      if result == 1:
        real_vitoria += 1
      else:
        real_contra += 1

      if prob > 0.5:
        prob_vitoria_mandante.append(prob)
        prob_contra_mandante.append(1-prob)
        sugere_vitoria += 1

        if result == 1:
          l_p_lista.append(np.round(100 * (oddH - 1),2))
        else:
          l_p_lista.append(-100)
      else:
        sugere_contra += 1
        prob_contra_mandante.append(prob)
        prob_vitoria_mandante.append(1-prob)

        if result == 0:
          l_p_lista.append(np.round(100 * (oddLayH - 1),2))
        else:
          l_p_lista.append(-100)
    
    df_base_predicoes_02['L/P'] = l_p_lista

    # Últimas etapas:
    temp_df = concatena_rodada_teste_ao_treino(temp_df,df_rodada_teste)
    temp_df_02 = concatena_rodada_teste_a_rodada_anterior(temp_df_02, df_base_predicoes_02)

    rodada += 1

  media_probs_vitoria_madante = np.round(np.mean(prob_vitoria_mandante),2)
  media_probs_contra_mandante = np.round(np.mean(prob_contra_mandante),2)
  std_vitoria_mandante = np.round(np.std(prob_vitoria_mandante),2)
  std_contra_mandante = np.round(np.std(prob_contra_mandante),2)

  prop_vitoria_mandante_sugerida = str(np.round((sugere_vitoria/(sugere_vitoria+sugere_contra))*100,2))+"%"
  prop_vitoria_mandante_real = str(np.round((real_vitoria/(real_vitoria+real_contra))*100,2))+"%"
  
  proporcao.append([prop_vitoria_mandante_real, prop_vitoria_mandante_sugerida, str(media_probs_vitoria_madante), str(media_probs_contra_mandante), str(std_vitoria_mandante), str(std_contra_mandante)])

  temp_df_02.to_excel("/content/drive/MyDrive/Iniciação Científica - UAEst - Prof. Alexsandro_2019.2/Pesquisa_Caio Medeiros/Projeto 2020_2021_Deep Learning/Resultados/Proporções/Analise Exploratoria_probs/Analise_Explo_Temporada_"+str(temporada)+"_"+str(camadas)+'_com_'+ str(neuronios)+'_neuronios.xlsx')

  sugere_vitoria = 0
  sugere_contra = 0
  real_vitoria = 0
  real_contra = 0
  prob_vitoria_mandante = []
  prob_contra_mandante = []
  temporada += 1
  teste += 1
  treino_base += 1

temps = ['2013','2014','2015','2016','2017','2018','2019']
cols = ['PRVM','PSVM','MdPbVM','MdPbDM','StdVM','StdDM']
rows = np.array(proporcao)
df = pd.DataFrame(rows,columns=cols,index = temps)
writer = '/content/drive/MyDrive/Iniciação Científica - UAEst - Prof. Alexsandro_2019.2/Pesquisa_Caio Medeiros/Projeto 2020_2021_Deep Learning/Resultados/Proporções/Proporção_'+str(camadas)+'_com_'+str(neuronios)+'_neuronios.xlsx'
df.to_excel(writer)

ValueError: ignored

In [ ]:
display(temp_df_02)